In [58]:
import pandas as pd
people = pd.read_csv('people.csv')
import datetime
import requests
import reverse_geocoder as rg 
import pprint 
#date_object = datetime.date.today()
#print(date_object)

In [22]:
def clean_people(df):
    # rename columns:
    df = df.rename(columns={'email address': 'email'})
    
    # remove rows which have an empty "first_name" (NA):
    #df = df[df.first_name.notna()] <- equivalent to next line:
    df = df.dropna(subset=['first_name'])
    
    # drop duplicates on ID column:
    df = df.drop_duplicates()
    
    # Normalize gender column:
    df['gender'] = df['gender'].replace({'Female': 'F', 'Male': 'M'})
    
    # Convert column "age" to number (coerce: put NaN for bad values):
    df['age'] = pd.to_numeric(df.age, errors='coerce')
    
    # Convert columns to date type:
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    df['ls']=df['last_seen'].apply(lambda s: pd.Timestamp(s).date())
    # When missing, last seen should fallback to the registration date:
    df['last_seen'] = df.last_seen.combine_first(df.registration)
    
    # Add a "full_name" column by concatenating two other ones:
    df['full_name'] = df.first_name + " " + df.last_name
    
    # Add a "country" column by extracting it from the address, with a split:
    df['country'] = df.address.str.split(', ').str[1]
    
    # Column "money" contains values like "$50.23" or "€23,09".
    # We want to make it uniform (only dollar currency) and as number, not str.
    df['currency'] = df.money.str[0]  # extract first char ($/€) to a new "currency" column
    df['money'] = df.money.str[1:].str.replace(',', '.')  # extract remaining chars and replace , by .
    df['money'] = pd.to_numeric(df.money)  # convert to number
    # convert euros cells to dollar:
    df.loc[df.currency == '€', 'money'] = df[df.currency == '€'].money * 1.10
    del df['currency']  # remove "currency" column which is now useless
    
    # Keep only rows where email is not NA:
    df = df.dropna(subset=['email'])
    # Keep only rows where email is a good email:
    # CAUTION: in the real world you should not use dummy regexes like this to validate email addresses,
    # but instead use a dedicated tool like https://github.com/syrusakbary/validate_email.
    df = df[df.email.str.contains('.+@[0-9a-zA-Z\.\-_]+\.\w{2,}')]
    # Some users may use email alias (example: john.smith+truc@gmail.com is an alias for john.smith@gmail.com).
    # We want to drop these duplicates. To do that, we extract the 'alias' part with a regex:
    groups = df.email.str.extract('([0-9a-zA-Z\.\-_]+)(\+[0-9a-zA-Z\.\-_]+)?(@[0-9a-zA-Z\.\-_]+\.\w{2,})')
    df['email'] = groups[0] + groups[2]  # we override the email with the email without the alias part
    # Then, just use drop_duplicates, which will keep the first line by default:
    df = df.drop_duplicates(subset=['email'])
    
    return df

df_clean = clean_people(people)

In [23]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,ls,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,2019-06-03,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,2019-08-30,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,2019-03-27,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,2019-03-12,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,2018-12-26,Gordie Bodicum,Czech Republic
6,27539,Dulcine,Stopforth,dstopforth31@forbes.com,F,22.0,13.860,-35.587670,-6.255393,NaN,2017-12-13,False,2019-01-15 15:39:05,"Serrinha, Brazil",entrée/plat/dessert,2019-01-15,Dulcine Stopforth,Brazil
7,27469,Denny,Hasnip,dhasnip13@so-net.ne.jp,F,47.0,83.567,-89.917591,14.113818,NaN,2017-05-01,False,2019-03-08 18:46:46,"Comapa, Guatemala",entrée/plat/dessert,2019-03-08,Denny Hasnip,Guatemala
9,27565,Esmeralda,Hollindale,ehollindale3r@guardian.co.uk,F,29.0,60.590,-36.508738,-54.281149,NaN,2017-10-05,False,2018-04-12 15:45:51,"Grytviken, China",entrée/plat/dessert,2018-04-12,Esmeralda Hollindale,China
10,27446,Skippie,Arlott,sarlottg@so-net.ne.jp,M,49.0,27.797,38.954230,55.877840,NaN,2017-08-13,False,2018-10-06 10:55:59,"Malaya Dubna, Russia",entrée/plat/dessert,2018-10-06,Skippie Arlott,Russia
11,27550,Shermie,Locard,slocard3c@lycos.com,M,65.0,35.948,112.813530,-7.305958,NaN,2018-09-29,False,2019-05-03 00:23:45,"Wonorejo, Indonesia",entrée/plat/dessert,2019-05-03,Shermie Locard,Indonesia


In [24]:
def inactivite(datels):
    inactivite = False
    if (datels<datetime.date.today()):
        inactivite = True
    return inactivite


df_clean["inactive"] = df_clean["ls"].apply(lambda s: inactivite(s))
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,ls,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,True,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,2019-06-03,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,True,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,2019-08-30,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,True,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,2019-03-27,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,True,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,2019-03-12,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,True,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,2018-12-26,Gordie Bodicum,Czech Republic
6,27539,Dulcine,Stopforth,dstopforth31@forbes.com,F,22.0,13.860,-35.587670,-6.255393,NaN,2017-12-13,True,2019-01-15 15:39:05,"Serrinha, Brazil",entrée/plat/dessert,2019-01-15,Dulcine Stopforth,Brazil
7,27469,Denny,Hasnip,dhasnip13@so-net.ne.jp,F,47.0,83.567,-89.917591,14.113818,NaN,2017-05-01,True,2019-03-08 18:46:46,"Comapa, Guatemala",entrée/plat/dessert,2019-03-08,Denny Hasnip,Guatemala
9,27565,Esmeralda,Hollindale,ehollindale3r@guardian.co.uk,F,29.0,60.590,-36.508738,-54.281149,NaN,2017-10-05,True,2018-04-12 15:45:51,"Grytviken, China",entrée/plat/dessert,2018-04-12,Esmeralda Hollindale,China
10,27446,Skippie,Arlott,sarlottg@so-net.ne.jp,M,49.0,27.797,38.954230,55.877840,NaN,2017-08-13,True,2018-10-06 10:55:59,"Malaya Dubna, Russia",entrée/plat/dessert,2018-10-06,Skippie Arlott,Russia
11,27550,Shermie,Locard,slocard3c@lycos.com,M,65.0,35.948,112.813530,-7.305958,NaN,2018-09-29,True,2019-05-03 00:23:45,"Wonorejo, Indonesia",entrée/plat/dessert,2019-05-03,Shermie Locard,Indonesia


In [25]:
df_clean = df_clean.dropna(subset=['phone'])
df_clean = df_clean[df_clean.phone.str.contains('^[0-9]{8,}$')]
df_clean["portable"] = df_clean.phone.str.startswith("^0[6-7]") 
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,ls,full_name,country,portable
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,True,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,2019-06-03,Leandra Pabelik,Czech Republic,False
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,True,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,2018-12-26,Gordie Bodicum,Czech Republic,False
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,False,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,NaT,Stanley McCullen,China,False
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,True,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,2019-05-04,Shelley Skeemor,Indonesia,False
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,NaT,Mort Midford,Nigeria,False
20,27462,Graig,Brownsea,gbrownseaw@arstechnica.com,M,54.0,80.230,110.393439,-7.213328,0262855559,2018-05-22,True,2019-09-28 13:28:10,"Sidomukti, Indonesia",entrée/plat/dessert,2019-09-28,Graig Brownsea,Indonesia,False
24,27563,Andre,Lugsdin,alugsdin3p@microsoft.com,M,30.0,40.630,12.219712,57.121813,0144026028,2017-01-15,True,2019-06-03 02:11:38,"Varberg, Sweden",entrée/plat/dessert,2019-06-03,Andre Lugsdin,Sweden,False
27,27551,Brigg,Bryett,bbryett3d@prlog.org,M,20.0,28.050,102.638296,24.301245,0746863267,2017-10-25,True,2019-04-10 01:46:26,"Dayingjie, China",entrée/plat/dessert,2019-04-10,Brigg Bryett,China,False
30,27440,Erick,Meharry,emeharrya@wikispaces.com,M,52.0,44.792,3.575175,47.794456,0098166909,2017-04-21,True,2018-07-11 08:22:11,"Auxerre, France",entrée/plat/dessert,2018-07-11,Erick Meharry,France,False
31,27593,Evelyn,Olenichev,eolenichev4j@tripadvisor.com,F,44.0,54.252,23.711756,37.944288,0214068650,2017-01-02,True,2019-08-07 22:59:39,"Néa Smýrni, Greece",entrée/plat/dessert,2019-08-07,Evelyn Olenichev,Greece,False


In [26]:
df_clean.is_inactive=(df_clean.last_seen<datetime.date.today())

/home/sonia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.
/home/sonia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [134]:
#on extrait le nom de la ville pour vérifier avec celui des coordonnées GPS
df_clean["ville"]=df_clean.address.apply(lambda s: s.split(",")[0])
df_clean.head(5)

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,...,preference,ls,full_name,country,portable,coordinates,coor_GPS_ville,ville,Same_address,test
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.18,18.284100,49.632552,0136319724,...,entrée/plat/dessert,2019-06-03,Leandra Pabelik,Czech Republic,False,"(49.6325518, 18.2841004)",None,Palkovice,False,None
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.26,13.978681,49.786243,0146943857,...,entrée/plat/dessert,2018-12-26,Gordie Bodicum,Czech Republic,False,"(49.786243299999995, 13.978680800000001)",None,Jince,False,None
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,...,entrée/plat/dessert,NaT,Stanley McCullen,China,False,"(28.074648999999997, 119.141473)",None,Longquan,False,None
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.50,111.878471,-7.584917,0547556034,...,entrée/plat/dessert,2019-05-04,Shelley Skeemor,Indonesia,False,"(-7.5849174, 111.8784707)",None,Guyangan,False,None
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,...,entrée/plat,NaT,Mort Midford,Nigeria,False,"(5.9592535, 6.8485009)",None,Ozubulu,False,None


In [72]:
#pip install reverse_geocoder
#pip install pprint
lat='36.858610299999995'
lon= '10.1510866'

def flatten(l):
    for el in l:
        if isinstance(el, list):
            yield from flatten(el)
        else:
            yield el


def reverseGeocode(coordinates): 
    c=""
    result = rg.search(coordinates)
    print([d["name"] for d in flatten(result)])

    #pprint.pprint([d["name"] for d in flatten(result)])  
      
    # result is a list containing ordered dictionary. 
  
# Driver function 
if __name__=="__main__": 
      
    # Coorinates tuple.Can contain more than one pair. 
    coordinates =[lat, lon]
    reverseGeocode(coordinates)
    

['Tunis']


In [133]:
#on crée colonne cordinates contenant (lat,lon)
df_clean['coordinates']=list(zip(df_clean.lat, df_clean.lon))


#df_clean['coor_GPS_ville']=[reverseGeocode(coordinates) for coordinates in df_clean['coordinates']]
df_clean['coor_GPS_ville']=df_clean['coordinates'].apply(lambda x: reverseGeocode(x))


#On crée une colonne qui renvoie un bool pour comparer entre la valeur de la ville adresse et la 
# valeur de la ville obtenue par les coordonnées GPS
df_clean["Same_address"] = (df_clean['coor_GPS_ville'] == df_clean[["ville"]])

['Palkovice']
['Jince']
['Anren']
['Nganjuk']
['Ozubulu']
['Ambarawa']
['Varberg']
['Dayingjie']
['Auxerre']
['Nea Smyrni']
['Trogan Barat']
['Saint-Savin']
['Overum']
['Kopychyntsi']
['Xibu']
['Araci']
['Kajan']
['Serpong']
['Malogoszcz']
['Kozje']
['Ceper']
['Kriel']
['Beziers']
['Khadyzhensk']
['Chaparral']
['Riverton']
['Neuquen']
['Karlskoga']
['Aracatuba']
['Saint-Maximin-la-Sainte-Baume']
['Listowel']
['Krajan Sawahan']
['Patsi']
['Citeureup']
['Dos Quebradas']
['Sao Roque']
['Kazimierz Dolny']
['Liulinzhou']
['Kcynia']
['Bokaa']
['Aliwal North']
['Skrunda']
['Passage West']
['Saniwonorejo']
['Petrolina de Goias']
['Maoping']
['Pizhou']
['Mogoditshane']
['Xiaoweizhai']
['Zharkovskiy']
['Makubetsu']
["Novaya Usman'"]
['Banjar Karangsuling']
['Nanyang']
['Jieduo']
['Cimenga']
['Bergeforsen']
['Carbajales']
['Kashmar']
['Hullo']
['Kleszczow']
['Bantul']
['Stefan Voda']
['Wudil']
['Wiesbaden']
['Inirida']
['Ganshui']
['Vanersborg']
['Jhang Sadr']
['Mantingantengah']
['Indaiatuba']
